# CONVOLUTIONAL NEURAL NETWORK NB (CNN-First on DL)  

## libraries and env configuration 


In [1]:
#packages from tensor flow
import tensorflow as Tf

# tensor flow for optimizing the model 


# basis packages 
import os as os 
import matplotlib.pyplot as plt
import numpy as np
import scipy 
import ssl

In [2]:
physicalDevice = Tf.config.experimental.list_physical_devices('GPU')
Tf.config.experimental.set_memory_growth(physicalDevice[0], True)
print("Num GPUs Available and used :  ", len(physicalDevice))


Num GPUs Available and used :   1


In [3]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2' # Suppress TensorFlow logging (1)	

## data preprocessing 

In [4]:
# Load MNIST dataset
(xTrainMnist,yTrainMnist),(xTestMnist,yTestMnist)=Tf.keras.datasets.mnist.load_data()

### normalization 

In [5]:
xTrainMnist.max()

255

In [6]:
xTestMnist.max()


255

In [ ]:
xTestMnist = xTestMnist.astype('float32') / 255 

#### changing normalization for data train: 

In [ ]:
# we normalize tghe data lessing the mean and the standar deviation to xtrain data in order to have a better performance on the training process 

MnistMean = xTrainMnist.mean()
MnistStd = xTrainMnist.std()

# normalize the data dividing by the sd assecuring the none zero value of the sd

xTrainMnist = (xTrainMnist-MnistMean)/(MnistStd+1e-7)

# also normalize the test data using mean and std from training data cause the idea is that the network doesnt know these parameters of the test set

xTestMnist = (xTestMnist - MnistMean)/(MnistStd+1e-7)

""" also could use : 
mean = np.mean(xTrainMnist)
print(mean)
"""

### split train data

In [8]:
xTrainMnist, xValidMnist = xTrainMnist[5000:],xTrainMnist[:5000]
yTrainMnist, yValidMnist = yTrainMnist[5000:],yTrainMnist[:5000]

look dimension size

In [9]:
xTrainMnist.shape, yTrainMnist.shape, xValidMnist.shape, yValidMnist.shape

((50000, 28, 28), (50000,), (5000, 28, 28), (5000,))

### data argumentation for best performance

In [ ]:
datagen = Tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = True,
    vertical_flip = True
    )

### see number of classes/labels in order to binarize 

In [14]:
len(np.unique(yTrainMnist)) # 10 classes

10

####  binarizing the labels in order to use only categorical cross entropy without sparse 


In [15]:
yTrainMnist = Tf.keras.utils.to_categorical(yTrainMnist)
yTestMnist = Tf.keras.utils.to_categorical(yTestMnist)


In [16]:
yTrainMnist[0] # one hot encoding of the labels

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [17]:
print(yTrainMnist[0:10]) # display the first 10 labels of the training set
print(yTestMnist[0:10]) # display the first 10 labels of the test set

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [13]:
xTrainMnist.shape, yTrainMnist.shape, xTestMnist.shape, yTestMnist.shape

((50000, 28, 28), (50000,), (10000, 28, 28), (10000,))

## creating a sequential model 

In [ ]:
BasicMnistModel = Tf.keras.models.Sequential() 
# Add a Flatten layer to convert the 2D images to 1D vectors  ( transforming my 28 x 28 tensor to a 1D array ) 
BasicMnistModel.add(Tf.keras.layers.Flatten(input_shape=(28, 28))) 
BasicMnistModel.summary() # display the summary of the model 

In [ ]:
BasicMnistModel.add(Tf.keras.layers.Dense(256, activation ='relu')) # add a fully connected layer with 256 neurons and ReLU activation function     
BasicMnistModel.add(Tf.keras.layers.Dense(128, activation ='relu')) # add a fully connected layer with 128 neurons and ReLU activation function 
BasicMnistModel.add(Tf.keras.layers.Dense(10, activation ='softmax')) # add a fully connected layer with 10 neurons and softmax activation function ( output layer )    


In [ ]:
# proceed compiling the model 
BasicMnistModel.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics= ['accuracy']) # use Adam optimizer and sparse categorical crossentropy loss function

### proceding training the model 

In [ ]:
# fit the model to the training data
MnistHistory = BasicMnistModel.fit(xTrainMnist, yTrainMnist, epochs = 5 , batch_size = 32, validation_split = 0.3, validation_data=(xTestMnist, yTestMnist), verbose = 1) # fit the model to the training data with 5 epochs and a batch size of 32, using 30% of the training data for validation and displaying the progress bar

### lets see accuracy of the Mnist model : 


In [ ]:
loss, acc = BasicMnistModel.evaluate(xTestMnist, yTestMnist) # evaluate the model on the test set


In [ ]:
plt.plot(MnistHistory.history['loss'], label='Loss (TRAIN)')
plt.plot(MnistHistory.history['val_loss'], label='Loss (VALIDATION)')
plt.xlabel('EPOCHS')
plt.ylabel('LOSS')
plt.title('Loss per epoch')
plt.legend()
plt.grid()
plt.show()
print(f" error w/h noisy at the entry of the model : {loss*100:.2f}%")


In [ ]:
plt.plot(MnistHistory.history['accuracy'], label='accuracy (TRAIN)')
plt.plot(MnistHistory.history['val_accuracy'], label='accuracy (VALIDATION)')
plt.xlabel('EPOCHS')
plt.ylabel('accuracy')
plt.title('accuracy per epoch')
plt.legend()
plt.grid()
plt.show()

print(f"accuracy w/h noisy at the entry of the model : {acc*100:.2f}%")


## proceed with a class prediciton for our random sample

use a random variable to our data sample 

In [ ]:
id = np.random.choice(xTestMnist.shape[0], 1000, replace=False) 


In [ ]:
MnistPredictions = BasicMnistModel.predict(xTestMnist[id[0]:id[0]+1]) # make predictions on the test set

In [ ]:
predicted_class = np.argmax(MnistPredictions, axis=1)
plt.imshow(xTestMnist[id[0]])

print("predicted class:" , predicted_class)
print("rial class: ",yTestMnist[id[0]]) # display the true class of the image